In [ ]:
from pyspark.sql import SparkSession, SQLContext

class ShudCrawler:
    def __init__(self):
        self.sparkSession = SparkSession.builder\
            .appName("shud")\
            .config("spark.some.config.option", "some-value")\
            .getOrCreate()
        self.sqlContext = SQLContext(self.sparkSession)    

        
        
    def mainProcess(self):
        """Initialize the processing"""
        initList = [("https://h_amazon.com/DealsHomeUrl", "false")]
        #initList.append(("https://x1_amazon.com/x1", "false"))
        #initList.append(("https://x2_amazon.com/x2", "false"))
        
        df = self.sparkSession.createDataFrame(initList, schema=["url", "parsed"])
        
        self.sqlContext.registerDataFrameAsTable(df, "UrlList")

        df = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
        
        '''
        print(len(df.rdd.collect()))
        for url in df.rdd.collect():
            self.parse(url[0])
            df.show()
            df2 = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
            df = df.union(df2).dropDuplicates()
            if indx > 10:
                break
            indx +=1
        '''
        
        indx = 0
        
        while len(df.rdd.collect()) > 0:
            print("Tour ", indx)
            for url in df.rdd.collect():
                self.parse(url[0], indx)                
            
            df = self.sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
            
            if indx > 3:
                break
            indx +=1
            
        
    def parse(self, url, indx):
        # Parse url and add newly found urls
        newUrls = []
        #newUrls.append(("https://x1_amazon.com/x1", "false"))
        if indx < 6:
            newUrls.append(("https://x" + str(indx) + "_amazon.com/x"+ str(indx), "false"))
        
        #Set current url as parsed
        newUrls.append((url, "true"))
        
        #Get all urls to synchronize and update
        df = self.sqlContext\
            .sql("SELECT url, parsed from UrlList where url <>'%s'" % url)\
            .union(self.sparkSession.createDataFrame(newUrls))\
            .union(self.sparkSession.createDataFrame([(url, "true")]))\
            .dropDuplicates(['url'])
        
        self.sqlContext.registerDataFrameAsTable(df, "UrlList")
        
        df.show()
        
        

#Testing
st = ShudCrawler()
st.mainProcess()


def x():
    '''df = df.union(spark.createDataFrame([("https://x1_amazon.com/xx", "false")]))
    df = df.union(spark.createDataFrame([("https://x2_amazon.com/xx", "false")]))
    df = df.union(spark.createDataFrame([("https://x1_amazon.com/xx", "false")]))
    #df.show()
    sqlContext.registerDataFrameAsTable(df, "UrlList")
    #tt = "table1" in sqlContext.tableNames()
    #df2 = sqlContext.sql("SELECT url, parsed from UrlList where parsed = 'false'")
    #df2.collect()
    l = ['x','y']
    df = spark.createDataFrame((l, 'false'), schema=["url", "parsed"])
    '''
    


In [2]:
from pyspark.sql import SparkSession, SQLContext
sparkSession = SparkSession.builder\
            .appName("ShudSparkSession")\
            .config("spark.some.config.option", "some-value")\
            .getOrCreate()
            
sqlContext = SQLContext(sparkSession) 
l = [(1, 'a'), (2, 'b'), (3, 'c')]
df = sparkSession.createDataFrame(l, schema=["id", "let"])
sqlContext.registerDataFrameAsTable(df, "Letters")

class classX():
    def __init__(self):
        self.sparkSession = SparkSession.builder\
            .appName("ShudSparkSession")\
            .config("spark.some.config.option", "some-value")\
            .getOrCreate()
            
        self.sqlContext = SQLContext(sparkSession) 
    def process(self):
        df = self.sqlContext.sql("SELECT id, let from Letters")
        df.show()
        
        
x = classX()
x.process()
        


+---+---+
| id|let|
+---+---+
|  1|  a|
|  2|  b|
|  3|  c|
+---+---+



In [ ]:
#Testing Shud Crawler
from pyspark.sql import SparkSession, SQLContext

sparkSession = SparkSession \
        .builder \
        .appName("ShudSparkSession") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
        
sqlContext = SQLContext(sparkSession)
df = sqlContext.sql("SELECT * from WorkTable")
df.show()

In [ ]:
from pyspark import SparkContext
import str

sc = SparkContext('local[2]', 'pyspark tutorial') 

lines = sc.textFile(response.url)
lines.union(sc.textFile(response.url))

def noPunctuations(text):
    """Removes punctuation and convert to lower case
    Args:
        text (str): A string.
    Returns:
        str: The cleaned up string.
    """
    return text.translate(str.maketrans("","",string.punctuation)).lower()

lines_rdd = sc.textFile(get(1), 1)
counts = lines_rdd.map(noPunctuations).flatMap(lambda x: x.split(' ')) \
         .map(lambda x: (x, 1)) \
		 .reduceByKey(lambda x, y: x+y)
		 
for (word, count) in counts.collect():
   print(word,count)